참고한 블로그:
https://yunwoong.tistory.com/297

TextGenerativeModel document: https://cloud.google.com/vertex-ai/docs/generative-ai/sdk-for-llm/sdk-use-text-models

https://console.cloud.google.com/welcome/new?project=ybigta-414314

In [1]:
!pip install "google-cloud-aiplatform>=1.38"

In [2]:
import os
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from typing import List, Optional

In [3]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/ybigta-414314-448985d72b01.json"

In [4]:
project_id = "ybigta-414314"
location = "asia-northeast3" #한국 서울

In [5]:
vertexai.init(project=project_id, location=location)

In [ ]:
!gcloud auth login

In [ ]:
from vertexai.language_models import TextGenerationModel


model = TextGenerationModel.from_pretrained("text-bison@002")


#test
print(model.predict(
    "What is the best recipe for banana bread? Recipe:",
    # The following are optional parameters:
    #max_output_tokens=128,
    #temperature=0,
    #top_p=1,
    #top_k=5,
))

In [10]:
# !gcloud config set project ybigta-414314

Updated property [core/project].


In [ ]:
from vertexai.preview.generative_models import GenerativeModel
model = GenerativeModel("gemini-pro")

In [ ]:
def summarize(news_article: str) -> str:

  #Generate a summarization of the given news.
  model = TextGenerationModel.from_pretrained("text-bison@002")
  prompt = f"Summarize the following financial news article:\n\n{news_article}"

  summary = model.predict(
      prompt,
      max_output_tokens=150,
      temperature=0.7,
      top_p=0.9,
      top_k=10)

  return summary.text

In [ ]:
def detect_economic_terms(news_article: str) -> List[str]:
    """
    Tries to detect economic terms in a financial news article using the TextGenerationModel.

    Parameters:
    - news_article (str): The financial news article to be analyzed.

    Returns:
    - List[str]: List of detected economic terms.
    """
    # Build a prompt for economic term detection
    prompt = f"""Identify economic terms in the following financial news article.
                  Example 1
                  Input:
                  Emerging markets soared more than 33 percent in 2017, and Todd Gordon of TradingAnalysis.com says the rally won't stop. A big part of the rally in emerging markets, tracked by the emerging market ETF EEM , was a weak dollar. And given that Gordon still sees the inverse relationship between EEM and the dollar, measured in his charts by the dollar-tracking ETF UUP , he believes the U.S. currency will continue to help the group. "We have a falling U.S. dollar, which will support international and emerging market currencies and will give those EEM stocks a boost," Gordon said Tuesday on CNBC's "Trading Nation." The U.S. dollar in 2017 posted its worst annual performance in 14 years, while EEM saw its best performance since 2013. As for how high the latter could go, Gordon says EEM has broken "resistance" at around $45, which was the ETF's 2014 highs. That $45 region is now what he calls "support," and he sees it rallying to $50, which the ETF hasn't hit since mid-2011. To play for a move higher, Gordon suggested buying the February 48/50 call spread for 72 cents, or $72 per options contract. This means that if EEM closes above $50 on Feb. 16, then Gordon could make a maximum reward of $128 on the trade. But if EEM were to close below $48, then Gordon would lose the $72 he paid for the trade. As a result, Gordon wants to establish a point at which to get out. "If the 72 cent premium we just laid out gets cut in half to about 36 cents, let's cut the trade and move on," he said. EEM started the year off strong, rallying more than 1 percent on Tuesday.
                  Output:

                  :\n\n{news_article}""" #prompt engineering 필요

    # Instantiate the TextGenerationModel
    model = TextGenerationModel.from_pretrained("text-bison@002") #NER 고려 필요

    # Generate content
    generated_content = model.predict(
        prompt,
        max_output_tokens=200,
        temperature=0.7,
        top_p=0.9,
        top_k=10
    )

    # Extract economic terms (simple example, may not be accurate)
    economic_terms = [term.strip() for term in generated_content.split()] #이 splitting이 가능하도록 prompt engineering 필요

    return economic_terms



SyntaxError: incomplete input (<ipython-input-10-07163a8828ef>, line 12)

In [ ]:
"def get_economic_term_definitions(economic_terms: List[str]) -> List[str]:

  # input에 원문, 요약본, term list가 한번에 들어가야 context가 반영될듯
    """
    Gets definitions for given economic terms using a TextGenerationModel.

    Parameters:
    - economic_terms (List[str]): List of economic terms.

    Returns:
    - List[str]: List of definitions for each economic term.
    """
    # Instantiate the TextGenerationModel
    model = TextGenerationModel.from_pretrained("text-bison@002")

    # Build a prompt for definitions of multiple economic terms
    prompt = f"Define the following economic terms: {', '.join(economic_terms)}"

    # Generate content using the prompt
    generated_content = model.predict(
        prompt,
        temperature=0.7,
        top_p=0.9,
        top_k=10
    )

    # Split the generated content into definitions for each term
    definitions = [definition.strip() for definition in generated_content.split('\n')]

    return definitions

In [ ]:
def start(news_article: str) -> None:
  summary = summarize(news_article)
  terms = detect_economic_terms(news_article)
  definitions = get_economic_term_definitions(terms)

  return summary, terms, definitions


In [ ]:
news = input("news article: ")
summary, terms, definitions = start(news)

##TODO
1. zero-shot, one-shot, few-shot 시도
2. `detect_economic_terms()`과 `get_economic_term_definitions()`에서 splitting이 가능하도록 output이 나오게 하기
3.  `detect_economic_terms()`에서 NER 고려해보기.
4. `get_economic_term_definitions()`을 TextGenerativeModel에게 시키는 것이 맞을까?
5. User의 input과 내 코드가 어떻게 연결되는지 물어보기
6. 그걸 알아낸 다음 User가 input 했을 때 모든 function이 작동하도록 연결시키기 (완)
7. `get_economic_term_definitions()`에 TextGenerativeModel을 여전히 사용할 거면, input으로 기사 원문, 요약본도 같이 넘겨주기

Crawling에서 궁금한 점:
- subscribe를 해야 볼 수 있는 기사를 크롤링으로 보는 건 불법 아닌가요?
- 불가능하다면 유저가 직접 복붙하는 방법은 어떤가요?

In [ ]:
!pip install spacy

In [ ]:
#NER 사용

import spacy
from vertexai.language_models import TextGenerationModel

def detect_economic_terms(news_article: str) -> List[str]:
    """
    Detects economic terms in a financial news article.

    Parameters:
    - news_article (str): The financial news article.

    Returns:
    - list: List of detected economic terms.
    """
    # Simple example: Use spaCy for named entity recognition
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(news_article)

    # Extract economic terms (e.g., organizations or monetary values)
    economic_terms = [entity.text for entity in doc.ents if entity.label_ in ["ORG", "MONEY"]]

    return economic_terms